In [ ]:
#r "BoSSSpad.dll"
// #r ".\binaries\XNSEC.dll"
// #r "C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
// #r "C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\XNSEC.dll"
// #r "C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
// #r "C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\XNSEC.dll"
// #r "C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
//#r "C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\XNSEC.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
Init();

In [ ]:
using BoSSS.Application.XNSEC;

In [ ]:
BoSSSshell.WorkflowMgm.Init("HeatedCavity_ConvergenceStudy");

In [ ]:
int NCellsFinestMesh = (int) Math.Pow(Math.Pow(2,5+1),2); 

In [ ]:
static var myDb = OpenOrCreateDatabase(@"C:\Databases\HeatedCavity_ConvergenceStudy3");

In [ ]:
BoSSSshell.WorkflowMgm.DefaultDatabase = myDb;

In [ ]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable;
BoSSSshell.WorkflowMgm.hConvergence.Update();

In [ ]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable; // The session table has to be created again in order to see the results from  hconvergence

## Error convergence study for Temperature

In [ ]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable;
var Tab2 = Tab.ExtractColumns("DGdegree:Temperature","Grid:hMin","Grid:NoOfCells" ,"L2Error_Temperature");
//Extract last point, the one against we are comparing our solutions with
Tab2 = Tab2.ExtractRows((iRow,RowEntries)=> Convert.ToInt32(RowEntries["Grid:NoOfCells"]) !=NCellsFinestMesh );

In [ ]:
var ErrorPlot = Tab2.ToPlot("Grid:hMin","L2Error_Temperature", "DGdegree:Temperature");
ErrorPlot.LogX = true;
ErrorPlot.LogY = true;
ErrorPlot.PlotNow()     // No semicolon!!

## Error convergence study for Velocity-X

In [ ]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable;
var Tab2 = Tab.ExtractColumns("DGdegree:Velocity*","Grid:hMin","Grid:NoOfCells" ,"L2Error_VelocityX");
//Extract last point, the one against we are comparing our solutions with
Tab2 = Tab2.ExtractRows((iRow,RowEntries)=> Convert.ToInt32(RowEntries["Grid:NoOfCells"]) !=NCellsFinestMesh );
var ErrorPlot = Tab2.ToPlot("Grid:hMin","L2Error_VelocityX", "DGdegree:Velocity*");
ErrorPlot.LogX = true;
ErrorPlot.LogY = true;
ErrorPlot.PlotNow()     // No semicolon!!

## Error convergence study for Velocity-Y

In [ ]:
var Tab = BoSSSshell.WorkflowMgm.SessionTable;
var Tab2 = Tab.ExtractColumns("DGdegree:Velocity*","Grid:hMin","Grid:NoOfCells" ,"L2Error_VelocityY");
//Extract last point, the one against we are comparing our solutions with
Tab2 = Tab2.ExtractRows((iRow,RowEntries)=> Convert.ToInt32(RowEntries["Grid:NoOfCells"]) !=NCellsFinestMesh );
var ErrorPlot = Tab2.ToPlot("Grid:hMin","L2Error_VelocityY", "DGdegree:Velocity*");
ErrorPlot.LogX = true;
ErrorPlot.LogY = true;
ErrorPlot.PlotNow()     // No semicolon!!

In [ ]:
ErrorPlot.Regression()

In [ ]:
var velX = ErrorPlot.ToGnuplot().PlotCairolatex(xSize:14,ySize:12);
velX.SaveTo(@"vel_Convergence.tex");